### Import libraries

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

### Read .csv

In [6]:
os.listdir()

['rick_diamonds.csv',
 'diamonds.csv',
 'README.md',
 '.gitignore',
 '.gitattributes',
 '.ipynb_checkpoints',
 '.git',
 'diamond_project.ipynb']

In [4]:
data = pd.read_csv('diamonds.csv')

In [7]:
rick = pd.read_csv('rick_diamonds.csv')

#### Save backup files

In [9]:
data_bk = data.copy()

In [10]:
rick_bk = rick.copy()

### Understand database

In [11]:
data.describe()

,carat,depth,table,price,x,y,z
count,48940.000000,48940.000000,48940.000000,48940.000000,48940.000000,48940.000000,48940.000000
mean,0.797817,61.751931,57.451161,3934.409644,5.730712,5.734333,3.538648
std,0.474126,1.430026,2.233450,3989.333861,1.121920,1.145344,0.706817
min,0.200000,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,61.000000,56.000000,949.000000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.690000,5.710000,3.520000
75%,1.040000,62.500000,59.000000,5331.250000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


2.70     699
2.71     682
2.69     676
2.68     654
2.72     630
        ... 
5.76       1
2.26       1
1.07       1
6.16       1
31.80      1
Name: z, Length: 370, dtype: int64